In [8]:
import dash
import dash_bootstrap_components as dbc
from dash import html as html
from dash import dcc as dcc
import pandas as pd
import plotly.express as px
import data_cleaning
import plotly.graph_objects as go
import numpy as np

# Load df dataset
df = data_cleaning.main()
df = data_cleaning.as_type(df, 'int', 'TOTAL_HOUSEHOLD_INCOME_-_CURRENT_DOLLARS')
df = data_cleaning.as_type(df, 'int', 'AGE_OF_RESPONDENT')
#create first figure
df_group = df.groupby('AGE_OF_RESPONDENT')['TOTAL_HOUSEHOLD_INCOME_-_CURRENT_DOLLARS'].mean().reset_index()
df_group = pd.DataFrame(df_group)
df_group = df_group[['AGE_OF_RESPONDENT', 'TOTAL_HOUSEHOLD_INCOME_-_CURRENT_DOLLARS']].sort_values(by='AGE_OF_RESPONDENT',ascending=True)
#new graph df
df['SEX_OF_RESPONDENT'] = df['SEX_OF_RESPONDENT'].astype(str)
#replace 1 with M and 2 with F
df['SEX_OF_RESPONDENT'] = df['SEX_OF_RESPONDENT'].replace({'1': 'M', '2': 'F'})
df.dropna(subset=['SEX_OF_RESPONDENT'], inplace=True)
pivot = df.pivot_table(index=['SURVEY_YEAR'], columns='SEX_OF_RESPONDENT', values='TOTAL_HOUSEHOLD_INCOME_-_CURRENT_DOLLARS', aggfunc='mean')
#reset the index
pivot = pivot.reset_index()
#drop rows where SEX is blank or not M or F
df = df[df['SEX_OF_RESPONDENT'].isin(['M', 'F'])]
#convert INCOME column to numeric type
#create second figure
df['PERSONAL_FINANCES_B/W_5_YEAR_AGO'] = df['PERSONAL_FINANCES_B/W_5_YEAR_AGO'].astype(str)
#replace the financial condition number into str
df['PERSONAL_FINANCES_B/W_5_YEAR_AGO'] = df['PERSONAL_FINANCES_B/W_5_YEAR_AGO'].replace({'1':'Better now','3' :'Same', '5': 'Worse now','8': 'DK','9':'NA'})
df_2022 = df.query("SURVEY_YEAR == 2022")
df_2022_PAGO5 = df_2022['PERSONAL_FINANCES_B/W_5_YEAR_AGO'].value_counts().reset_index().rename(columns={'index': 'sub_cat_values', 'PERSONAL_FINANCES_B/W_5_YEAR_AGO': 'counts'})
df_2017 = df.query("SURVEY_YEAR == 2017")
df_2017_PAGO = df_2017['PERSONAL_FINANCES_B/W_YEAR_AGO'].value_counts().reset_index().rename(columns={'index': 'sub_cat_values', 'PERSONAL_FINANCES_B/W_YEAR_AGO': 'counts'})
#create next graph
df_NewColumn = df[['PERSONAL_FINANCES_B/W_YEAR_AGO','VEHICLE_BUYING_ATTITUDES', 'ECONOMY_BETTER/WORSE_YEAR_AGO', 'UNEMPLOYMENT_MORE/LESS_NEXT_YEAR','ECONOMY_BETTER/WORSE_NEXT_YEAR','DURABLES_BUYING_ATTITUDES']]
corr = df_NewColumn.corr()
corr_columns = ['PRICES_UP/DOWN_NEXT_YEAR', 'DURABLES_BUYING_ATTITUDES', 'HOME_BUYING_ATTITUDES', 'VEHICLE_BUYING_ATTITUDES']
corr_df = df[corr_columns]
corr_matrix = corr_df.corr()
annotations = np.round(corr_matrix.values, 2)
counts = df['EDUCATION:_COLLEGE_GRADUATE'].value_counts()
# Get the sizes and labels for the pie chart
sizes = [counts[0], counts[1]]
labels = ['Non-College Graduates', 'College Graduates']

c:\Users\e707088\OneDrive - Blue Cross Blue Shield of Michigan\Documents\Python\final_project\data_cleaning.py:13: DtypeWarning:

Columns (13,15,16,18,32,34,36,37,41,45,46,55,71,83,84,86,87,90,91,92,96,98,100,101,103,104,105,106,107,108,109) have mixed types. Specify dtype option on import or set low_memory=False.



In [53]:
# Create four static graphs
fig1 = px.line(df_group, x='AGE_OF_RESPONDENT', y='TOTAL_HOUSEHOLD_INCOME_-_CURRENT_DOLLARS')
#fig2 = px.bar(df_fin, x='AGE_OF_RESPONDENT', y=['PERSONAL_FINANCES_B/W_NEXT_YEAR', 'PERSONAL_FINANCES_B/W_YEAR_AGO'])
fig2 = px.pie(df_2022_PAGO5, values='counts', names='sub_cat_values',
        labels={'sub_cat_values': 'Financial Condition'},
        #labels={'ou': 'ere'},
        color_discrete_sequence=px.colors.qualitative.Plotly)
fig2.update_traces(textposition='inside', textinfo='percent+label')
fig2.update_layout(font=dict(size=18),
                showlegend=True)
fig3 = px.pie(df_2017_PAGO, values='counts', names='sub_cat_values',
        labels={'sub_cat_values': 'Financial Condition'},
        color_discrete_sequence=px.colors.qualitative.Plotly)
fig3.update_traces(textposition='inside', textinfo='percent+label')
fig3.update_layout(font=dict(size=18),
                showlegend=True)
fig4 = px.imshow(corr, x=corr.columns, y=corr.columns,
        color_continuous_scale='RdBu', zmin=-1, zmax=1, aspect='auto')
fig4.update_layout(
                title={
                    'text': 'Correlation Heatmap',
                    'y': 0.95,
                    'x': 0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'
                }
            )
#create double bar graph using plotly express
fig5 = px.bar(pivot, x='SURVEY_YEAR', y=['M', 'F'], barmode='group')
#set plot title and labels
fig5.update_layout(title='Year on Year Average Income by Sex and Category', xaxis_title='Year, Sex', yaxis_title='Average Income')
#figure 6
fig6 = px.imshow(corr_matrix.values,
        x=corr_columns, y=corr_columns,
        color_continuous_scale='blues',
        zmin=-1, zmax=1,
        labels=dict(x='', y=''))
# add annotations to the heatmap
for i in range(len(corr_columns)):
    for j in range(len(corr_columns)):
        fig6.add_annotation(x=corr_columns[i], y=corr_columns[j],
                        text=str(annotations[i][j]),
                        showarrow=False, font=dict(color='white'))
# update layout to show the color scale and adjust the margins
fig6.update_layout(title="Correlation Matrix",
                coloraxis_colorbar=dict(title='Correlation'),
                margin=dict(l=100, r=100, t=50, b=100))
#figure 7 pie chart
fig7 = px.pie(values=sizes, names=labels, title='Proportion of College Graduates in Respondents')
fig7.update_traces(textposition='inside', textinfo='percent+label')

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define dropdown options
dropdown_options = {
    'AGE_OF_RESPONDENT': 'Age',
    'REGION_OF_RESIDENCE': 'Region of Residence',
    'SEX_OF_RESPONDENT': 'Sex',
    'MARITAL_STATUS_OF_RESPONDENT': 'Marital Status'
}

# Define groupby options
groupby_options = {
    'EDUCATION_OF_RESPONDENT': 'Education level',
    'EDUCATION:_COLLEGE_GRADUATE': 'Education: college grad',
    'POLITICAL_AFFILIATION': 'Political Affiliation'
}

# Define function to create data table
def create_table(df):
    return dbc.Table.from_dataframe(df, striped=True, bordered=True, hover=True)

# Define app layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H5('df Dashboard', className='text-center'),
            html.Hr(),
            html.H4('Select X-Axis'),
            dcc.Dropdown(
                id='dropdown-x',
                options=[{'label': label, 'value': value} for value, label in dropdown_options.items()],
                value='AGE_OF_RESPONDENT'
            ),
            html.H4('Select Group By'),
            dcc.Dropdown(
                id='dropdown-groupby',
                options=[{'label': label, 'value': value} for value, label in groupby_options.items()],
                value='EDUCATION_OF_RESPONDENT'
            ),
        ], md=4),
        dbc.Col([
            dcc.Tabs([
                dcc.Tab(label='Table', value='table'),
                dcc.Tab(label='Graph', value='graph'),
            ], id='tabs', value='table'),
            html.Div(id='display-page'),
        ], md=8)
    ])
])

# Define callback to update display page
@app.callback(
    dash.dependencies.Output('display-page', 'children'),
    [dash.dependencies.Input('dropdown-x', 'value'),
     dash.dependencies.Input('dropdown-groupby', 'value'),
     dash.dependencies.Input('tabs', 'value')]
)
def display_page(x, groupby, tab):
    # Compute groupby
    if tab == 'table':
        # Create data table
        df_grouped = df.groupby(groupby)[x].describe().reset_index()
        table = create_table(df_grouped)
        # Return table
        return table
    elif tab == 'graph':
        layout = html.Div([
                html.H5('Personal Income by Age'),
                html.Div([
                    dcc.Graph(id='fig1',figure=fig1)
                ]),
                html.H5('Personal Income by Sex'),
                html.Div([
                    dcc.Graph(id='fig5',figure=fig5)
                ]),
                html.H5('Correlation between Inflation, home amount, vehile, unemployment, and business conditions'),
                html.Div([
                    dcc.Graph(id='fig4', figure=fig4)
                ]),
                html.H5('Correlation between Inflation and Buying Attitudes'),
                html.Div([
                    dcc.Graph(id='fig6', figure=fig6)
                ]),
                html.H5('Personal Finace in 2017 v 2022'),
                html.Div(children=[
                    dcc.Graph(id='fig2',figure=fig2, style={'width': '49%', 'display': 'inline-block'}),
                    dcc.Graph(id='fig3',figure=fig3, style={'width': '49%', 'display': 'inline-block'})
                ]),
                html.H5('Proportion of College Grads in Respondents'),
                html.Div([
                    dcc.Graph(id='fig7', figure=fig7)
                ]),
             ])
        return layout
        #Return a list of the four graphs
        #return dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)#, dcc.Graph(figure=fig3), dcc.Graph(figure=fig4)
if __name__ == '__main__':
    app.run_server(port=8888)

Dash is running on http://127.0.0.1:8888/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8888/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Apr/2023 20:24:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 20:24:11] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 20:24:11] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 20:24:11] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 20:24:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 20:24:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 20:24:14] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 20:24:14] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


In [ ]:
px.bar(df_fin, x='AGE_OF_RESPONDENT', y='PERSONAL_FINANCES_B/W_NEXT_YEAR')

NameError: name 'df' is not defined

In [6]:
import dash
import dash_bootstrap_components as dbc
import dash_html_components as html

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the sidebar
sidebar = html.Div(
    [
        dbc.Label("Select an option"),
        dcc.Dropdown(
            id="sidebar-dropdown",
            options=[
                {"label": "Option 1", "value": 1},
                {"label": "Option 2", "value": 2},
                {"label": "Option 3", "value": 3},
            ],
            value=1,
        ),
    ],
    style={"position": "fixed", "top": 50, "left": 10},
)

# Define the main content
content = html.Div(id="page-content", style={"margin-left": "250px", "padding": "30px"})

# Define the app layout
app.layout = html.Div([sidebar, content])

# Define the callback to update the content based on the sidebar selection
@app.callback(
    dash.dependencies.Output("page-content", "children"),
    [dash.dependencies.Input("sidebar-dropdown", "value")],
)
def update_content(value):
    if value == 1:
        # Return layout for option 1
        return html.Div("Option 1 selected")
    elif value == 2:
        # Return layout for option 2
        return html.Div("Option 2 selected")
    elif value == 3:
        # Return layout for option 3
        return html.Div("Option 3 selected")

if __name__ == "__main__":
    app.run_server(port=8888)

Dash is running on http://127.0.0.1:8888/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
